In [38]:
import pandas as pd
suffix = 'es'
results = pd.read_csv(f'experiments/results_{suffix}.csv')
results.head()

,player_1,player_2,winner,prompt
0,Rojo,Azul,Azul,Opción A: Rojo. Opción B: Azul. Seleccionado:
1,Rojo,Verde,Rojo,Opción A: Rojo. Opción B: Verde. Seleccionado:
2,Rojo,Amarillo,Amarillo,Opción A: Rojo. Opción B: Amarillo. Seleccionado:
3,Rojo,Morado,Rojo,Opción A: Rojo. Opción B: Morado. Seleccionado:
4,Azul,Rojo,Azul,Opción A: Azul. Opción B: Rojo. Seleccionado:


In [ ]:
# import pandas as pd
# import re

# def create_template(row):
#     p1 = row['player_1']
#     p2 = row['player_2']
#     prompt = row['prompt']

#     # Find which player appears first in the string
#     pos1 = prompt.find(p1)
#     pos2 = prompt.find(p2)

#     if pos1 < pos2:
#         first, second = p1, p2
#     else:
#         first, second = p2, p1

#     # Use regex with word boundaries to avoid partial matches
#     template = re.sub(rf'\b{re.escape(first)}\b', '{A}', prompt)
#     template = re.sub(rf'\b{re.escape(second)}\b', '{B}', template)
    
#     return template

# results['template'] = results.apply(create_template, axis=1)
# print(results['template'].nunique())
# results.drop(columns=['prompt'], inplace=True)
# results.to_csv(f'experiments/results_{suffix}.csv', index=False)

50


In [ ]:
# from src.pref_models import fit_bradley_terry


# colors = list(results['player_1'].unique())
# data = pd.DataFrame()
# rankings = []
# for i, template in enumerate(results['template'].unique()):
#     template_results = results[results['template'] == template]
#     data = pd.concat([data, template_results], ignore_index=True)
#     ranking, win_matrix = fit_bradley_terry(data, colors)
#     rankings.append(ranking.assign(iteration=i))

# rankings = pd.concat(rankings)
# rankings.to_csv(f'experiments/ranking_{suffix}.csv', index=False)
# rankings

,Color,BT_Score,iteration
0,Amarillo,2.000727e+00,0
1,Azul,1.276982e+00,0
2,Verde,3.343121e-08,0
3,Rojo,-1.276982e+00,0
4,Morado,-2.000727e+00,0
...,...,...,...
0,Amarillo,4.743378e-01,49
1,Azul,2.440818e-01,49
2,Rojo,7.733106e-02,49
3,Verde,-1.385192e-01,49
